In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('train (1).csv')

In [3]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [4]:
df['Ticket'] = df['Ticket'].apply(lambda x: len(x))
df['Name'] = df['Name'].apply(lambda x: len(x))

In [5]:
df['FamilySize'] = df.SibSp + df.Parch

In [6]:
df['IsAlone'] = df.FamilySize.apply(lambda x: 1 if x > 0 else 0)

In [7]:
df['Cabin'] = df['Cabin'].str[0]

In [8]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,IsAlone
0,1,0,3,23,male,22.0,1,0,9,7.2500,NaN,S,1,1
1,2,1,1,51,female,38.0,1,0,8,71.2833,C,C,1,1
2,3,1,3,22,female,26.0,0,0,16,7.9250,NaN,S,0,0
3,4,1,1,44,female,35.0,1,0,6,53.1000,C,S,1,1
4,5,0,3,24,male,35.0,0,0,6,8.0500,NaN,S,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,21,male,27.0,0,0,6,13.0000,NaN,S,0,0
887,888,1,1,28,female,19.0,0,0,6,30.0000,B,S,0,0
888,889,0,3,40,female,NaN,1,2,10,23.4500,NaN,S,3,1
889,890,1,1,21,male,26.0,0,0,6,30.0000,C,C,0,0


In [9]:
df.isna().any()

PassengerId    False
Survived       False
Pclass         False
Name           False
Sex            False
Age             True
SibSp          False
Parch          False
Ticket         False
Fare           False
Cabin           True
Embarked        True
FamilySize     False
IsAlone        False
dtype: bool

In [10]:
df['Age'] = df.Age.interpolate()

In [11]:
df['Embarked'] = df.Embarked.fillna(df.Embarked.mode()[0])

In [12]:
df.Cabin = df.Cabin.fillna('Unknown')

In [13]:
dummies = pd.get_dummies(df[['Sex','Cabin','Embarked']])

In [14]:
df = pd.concat([df,dummies],axis = 1)

In [15]:
df = df.drop(columns = ['Name','PassengerId'])

In [16]:
Y = df[['Survived']]
X = df.drop(columns = ['Survived','Sex','Cabin','Embarked'])

In [17]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

X = sc.fit_transform(X)

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.25, random_state = 42, stratify = Y)

In [19]:
from sklearn.linear_model import LogisticRegression

log = LogisticRegression()

log.fit(X_train,Y_train)

C:\Users\tashi\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


LogisticRegression()

In [20]:
log.score(X_test,Y_test)

0.7892376681614349

In [23]:
X_new = pd.concat([pd.DataFrame(X),pd.DataFrame(log.predict(X))],axis = 1)

In [28]:
Xn_train, Xn_test, Yn_train, Yn_test = train_test_split(X_new,Y,test_size = 0.25, random_state = 42, stratify = Y)

In [29]:
from sklearn.ensemble import AdaBoostClassifier

adb = AdaBoostClassifier()

adb.fit(Xn_train,Y_train)
adb.score(Xn_train,Y_train)

C:\Users\tashi\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


0.842814371257485

In [30]:
adb.score(Xn_test,Y_test)

0.7623318385650224